# The `eensight` functionality for encoding categorical and numerical features

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.pipeline import make_pipeline
from sklearn.datasets import make_friedman1
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures

In [3]:
from eensight.utils import add_constant, tensor_product

from eensight.features.encode import IdentityEncoder, TrendFeatures
from eensight.features.encode import DatetimeFeatures, CyclicalFeatures, MMCFeatures
from eensight.features.encode import SafeOrdinalEncoder, SafeOneHotEncoder
from eensight.features.encode import TargetClusterEncoder, CategoricalEncoder
from eensight.features.encode import ICatEncoder, SplineEncoder, ISplineEncoder
from eensight.features.encode import ProductEncoder, ICatLinearEncoder, ICatSplineEncoder

## Utility encoders

### `eensight.features.encode.IdentityEncoder` 

The identity encoder returns what it is fed.

    Parameters
    ----------
    feature : str or list of str, default=None
        The name(s) of the input dataframe's column(s) to return.
        If None, the whole input dataframe is returned.
    include_bias : bool, default=False
        If True, a column of ones is added to the output.

We can create a dummy dataset:

In [ ]:
data = pd.DataFrame(index=pd.date_range(start='1/1/2018', end='31/12/2019', freq='H'))
data['day'] = data.index.dayofyear
data['x1'] = 4 + 3*np.sin(data['day']/365*2*np.pi)
data['x2'] = 4 * np.sin(data['day']/365*4*np.pi+365/2)
noise = np.random.normal(0, 0.9, len(data))
data['y'] = data['x1'] + data['x2'] + noise
data.head()

All encoders have a `n_features_out_` after fitting:

In [ ]:
enc = IdentityEncoder(feature='x1', include_bias=True)
assert enc.fit_transform(data).shape[1] == enc.n_features_out_
enc.n_features_out_

In [37]:
enc = IdentityEncoder(feature='x1')
assert np.allclose(enc.fit_transform(data), data[['x1']])

In [38]:
enc = IdentityEncoder(include_bias=True)
assert np.allclose(enc.fit_transform(data), add_constant(data))

## Creating time trend features 

### `eensight.features.encode.TrendFeatures` 

Generates time trend features.
    
    Parameters
    ----------
    feature : str, default=None
        The name of the input dataframe's column that contains datetime information.
        If None, it is assumed that the datetime information is provided by the
        input dataframe's index.
    include_bias : bool, default=False
        If True, a column of ones is added to the output.

In [ ]:
enc = TrendFeatures(include_bias=True)
features = enc.fit_transform(data)

with plt.style.context('seaborn-whitegrid'):    
    fig = plt.figure(figsize=(14, 3.54), dpi=96)
    layout = (2, 1)
    ax1 = plt.subplot2grid(layout, (0, 0))
    ax2 = plt.subplot2grid(layout, (1, 0))
    
    ax1.plot(features[:, 0])
    ax2.plot(features[:, 1])

In [40]:
assert enc.n_features_out_ == 2

## Adding date and time features

### `eensight.features.encode.DatetimeFeatures` 

Generates date and time features

    Parameters
    ----------
    feature : str, default=None
        The name of the input dataframe's column that contains datetime information.
        If None, it is assumed that the datetime information is provided by the
        input dataframe's index.
    remainder : str, :type : {'drop', 'passthrough'}, default='drop'
        By specifying ``remainder='passthrough'``, all the remaining columns of the
        input dataset will be automatically passed through (concatenated with the
        output of the transformer).
    replace : bool, default=False
        Specifies whether replacing an existing column with the same name is allowed
        (when `remainder=passthrough`).
    subset : str or list of str (default=None)
        The names of the features to generate. If None, all features will be produced:
        'month', 'week', 'dayofyear', 'dayofweek', 'hour', 'hourofweek', 'time'.
        The last 3 features are generated only if the timestep of the input's
        `feature` (or index if `feature` is None) is smaller than `pd.Timedelta(days=1)`. 

In [ ]:
enc = DatetimeFeatures(remainder='drop')
features = enc.fit_transform(data)
features.head()

In [42]:
assert features.shape[1] == enc.n_features_out_

In [ ]:
enc = DatetimeFeatures(remainder='passthrough')
features = enc.fit_transform(data)
features.head()

In [44]:
assert features.shape[1] == enc.n_features_out_

In [ ]:
enc = DatetimeFeatures(remainder='drop', subset=['month', 'dayofweek', 'hourofweek'])
enc.fit_transform(data)

## Encoding cyclical (seasonal) features

### `eensight.features.encode.CyclicalFeatures` 

Creates cyclical (seasonal) features as fourier terms

    Parameters
    ----------
    feature : str, default=None
        The name of the input dataframe's column that contains datetime information.
        If None, it is assumed that the datetime information is provided by the
        input dataframe's index.
    seasonality : str, default=None
        The name of the seasonality.
    period : float
        Number of days in one period.
    fourier_order : int
        Number of Fourier components to use.

**Note**: The encoder can provide default values for `period` and `fourier_order` if `seasonality` is one of `daily`, `weekly` or `yearly`.

In [ ]:
with plt.style.context('seaborn-whitegrid'):    
    fig = plt.figure(figsize=(14, 3.54), dpi=96)
    layout = (1, 1)
    ax = plt.subplot2grid(layout, (0, 0))
    
    data['y'].plot(ax=ax, alpha=0.5)

In [ ]:
enc = CyclicalFeatures(seasonality='yearly', fourier_order=3)
enc.fit(data)
enc.n_features_out_

Now let’s plot our transformed features:

In [ ]:
feature_trf = pd.DataFrame(data=enc.transform(data), index=data.index)

with plt.style.context('seaborn-whitegrid'):    
    fig, axs = plt.subplots(enc.n_features_out_, figsize=(14, 7), dpi=96)
    
    for i, col in enumerate(feature_trf.columns):
        feature_trf[col].plot(ax=axs[i])
    
fig.tight_layout()

Let's see how well this transformation works:

In [49]:
regr = LinearRegression().fit(feature_trf, data['y'])
pred = regr.predict(feature_trf)

In [ ]:
with plt.style.context('seaborn-whitegrid'):    
    fig = plt.figure(figsize=(14, 3.54), dpi=96)
    layout = (1, 1)
    ax = plt.subplot2grid(layout, (0, 0))
    
    data['y'].plot(ax=ax, alpha=0.5)
    pd.Series(pred, index=data.index).plot(ax=ax)

The root mean squared error is very close to the noise that was injected in the data (0.9):

In [ ]:
mean_squared_error(data['y'], pred, squared=False)

## Creating calendar features for metric learning

### `eensight.features.encode.MMCFeatures` 

Generates `month` and `dayofweek` one-hot encoded features for training an MMC
    (Mahalanobis Metric for Clustering) metric learning algorithm. 
    
    Parameters
    ----------
    month_feature : str, default='month'
        The name of the input dataframe's column that contains the `month` feature values.
    dow_feature : str, default='dayofweek'
        The name of the input dataframe's column that contains the `dayofweek` feature 
        values.

In [ ]:
enc = Pipeline([
    ('add_date_features', DatetimeFeatures(subset=['month', 'dayofweek'])),
    ('add_mmc_features',  MMCFeatures())
])


enc.fit_transform(data)

## Encoding categorical features

### A dataset to demonstrate the functionality

The dataset is related to the two-year historical log corresponding to years 2011 and 2012 from Capital Bikeshare system, Washington D.C., USA which is publicly available in http://capitalbikeshare.com/system-data. The dataset includes:

- `instant` : record index


- `dteday` : date


- `season` : season (1:springer, 2:summer, 3:fall, 4:winter)


- `yr` : year (0: 2011, 1:2012)


- `mnth` : month ( 1 to 12)


- `hr` : hour (0 to 23)


- `holiday` : weather day is holiday or not (extracted from http://dchr.dc.gov/page/holiday-schedule)


- `weekday` : day of the week


- `workingday` : if day is neither weekend nor holiday is 1, otherwise is 0.


- `weathersit` : 
    - 1: Clear, Few clouds, Partly cloudy, Partly cloudy
	- 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
	- 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
	- 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog


- `temp` : Normalized temperature in Celsius. The values are divided by 41 (max)


- `atemp`: Normalized feeling temperature in Celsius. The values are divided by 50 (max)


- `hum` : Normalized humidity. The values are divided by 100 (max)


- `windspeed` : Normalized wind speed. The values are divided to by (max)


- `casual` : count of casual users


- `registered` : count of registered users


- `cnt` : count of total rental bikes including both casual and registered

In [ ]:
path = 'play_data/bike_sharing/hour.csv'
data = pd.read_csv(path)
data = data.drop(['instant', 'casual', 'registered'], axis=1)
data.head()

In [20]:
X = data.drop('cnt', axis=1)
y = data['cnt']

### `eensight.features.encode.SafeOrdinalEncoder` 

Encodes categorical features as an integer array. The encoder converts the 
features into ordinal integers. This results in a single column of integers 
(0 to n_categories - 1) per feature.

It is implemented as a pipeline that connects a `sklearn.preprocessing.OrdinalEncoder` with a `sklearn.impute.SimpleImputer`.

    Parameters
    ----------
    feature : str or list of str, default=None
        The names of the columns to encode. If None, all categorical columns will
        be encoded.
    unknown_value : int, default=None
        This parameter will set the encoded value for unknown categories. It has to
        be distinct from the values used to encode any of the categories in `fit`.
        If None, the value `-1` is used. During `transform`, unknown categories
        will be replaced using the most frequent value along each column.

Let's fit on all columns

In [21]:
enc = SafeOrdinalEncoder()
enc = enc.fit(X)

In [ ]:
for feature in enc.features_:
    print('{0:10} is {1}'.format(feature, X.dtypes[feature]))

By default, the encoder regards as categorical any feature that is of type `object`, `bool`, `integer` or `category`.

Next, let's encode only the `weathersit` column, by first fitting on data that does not include `weathersit=4` and then transforming all data 

In [ ]:
print(f"The most frequent value is {X['weathersit'].mode().item()}")

In [29]:
enc = SafeOrdinalEncoder(feature='weathersit')
enc = enc.fit(X[X['weathersit'] != 4])

In [30]:
feature_trf = enc.transform(X)
feature_trf = pd.Series(data=feature_trf.squeeze(), index=X.index)

`weathersit=1` is mapped to 0, `weathersit=2` is mapped to 1, `weathersit=3` is mapped to 2, and `weathersit=4` is mapped where the most frequent value is mapped:

In [ ]:
print(f"weathersit=1 is mapped to: {feature_trf[X[X['weathersit']==1].index].unique().item()}")
print(f"weathersit=2 is mapped to: {feature_trf[X[X['weathersit']==2].index].unique().item()}")
print(f"weathersit=3 is mapped to: {feature_trf[X[X['weathersit']==3].index].unique().item()}")
print(f"weathersit=4 is mapped to: {feature_trf[X[X['weathersit']==4].index].unique().item()}")

### `eensight.features.encode.SafeOneHotEncoder` 

The encoder uses a `SafeOrdinalEncoder`to first encode the feature as an integer 
array and then a `sklearn.preprocessing.OneHotEncoder` to encode the features as 
an one-hot array. 
    
    Parameters
    ----------
    feature : str or list of str, default=None
        The names of the columns to encode. If None, all categorical columns will
        be encoded.
    unknown_value : int, default=None
        This parameter will set the encoded value of unknown categories. It has to
        be distinct from the values used to encode any of the categories in `fit`.
        If None, the value `-1` is used. During `transform`, unknown categories
        will be replaced using the most frequent value along each column.

In [34]:
enc = SafeOneHotEncoder(feature='weathersit')
enc = enc.fit(X[X['weathersit'] != 4])

In [ ]:
print(f'The encoder will transform the input column into {enc.n_features_out_} new ones')

In [36]:
feature_trf = enc.transform(X)
feature_trf = pd.DataFrame(data=feature_trf, index=X.index)

In [ ]:
print("The 1st column of the transformed feature corresponds to the "
      f"{X.loc[feature_trf.iloc[:, 0] == 1, 'weathersit'].unique()} categories")

print("The 2nd column of the transformed feature corresponds to the "
      f"{X.loc[feature_trf.iloc[:, 1] == 1, 'weathersit'].unique()} categories")

print("The 3rd column of the transformed feature corresponds to the "
      f"{X.loc[feature_trf.iloc[:, 2] == 1, 'weathersit'].unique()} categories")

### `eensight.features.encode.TargetClusterEncoder` 

Encodes a categorical feature as clusters of the target's values so that 
    to reduce its cardinality.

    Parameters
    ----------
    feature : str
        The name of the categorical feature to transform. This encoder operates 
        on a single feature.
    max_n_categories : int
        The maximum number of categories to produce.
    stratify_by : str or list of str (default=None)
        If not None, the encoder will first stratify the categorical feature into
        groups that have similar values of the features in `stratify_by`, and then
        cluster based on the relationship between the categorical feature and the
        target.
    excluded_categories : str or list of str (default=None)
        The names of the categories to be excluded from the clustering process. These
        categories will stay intact by the encoding process, so they cannot have the
        same values as the encoder's results (the encoder acts as an ``OrdinalEncoder``
        in the sense that the feature is converted into a column of integers 0 to
        n_categories - 1).
    unknown_value : int, default=None
        This parameter will set the encoded value of unknown categories. It has to
        be distinct from the values used to encode any of the categories in `fit`.
        If None, the value `-1` is used.
    min_samples_leaf : int, default=1
        The minimum number of samples required to be at a leaf node of the decision
        tree model that is used for stratifying the categorical feature if `stratify_by`
        is not None. The actual number that will be passed to the tree model is
        `min_samples_leaf` multiplied by the number of unique values in the categorical
        feature to transform.
    max_features : int, float or {"auto", "sqrt", "log2"}, default=None
        The number of features to consider when looking for the best split:
        - If int, then consider `max_features` features at each split of the decision tree.
        - If float, then `max_features` is a fraction and
        `int(max_features * n_features)` features are considered at each
        split.
        - If "auto", then `max_features=n_features`.
        - If "sqrt", then `max_features=sqrt(n_features)`.
        - If "log2", then `max_features=log2(n_features)`.
        - If None, then `max_features=n_features`.
    random_state : int, RandomState instance or None, default=None
        Controls the randomness of the estimator. To obtain a deterministic behaviour
        during fitting, ``random_state`` has to be fixed to an integer.

**Note**:
    This encoder does not replace unknown values with the most frequent one during
    `transform`. It just assigns them the value of `unknown_value`.

We can treat the `hr` feature as categorical and see if we can lump together some of its distinct values so that we reduce their number from 24 (hours of day) to 4. 

First, we can look at how the target changes for each `hr` value:

In [ ]:
to_group = pd.concat((y, X['hr']), axis=1)
grouped_mean = to_group.groupby('hr').mean()

with plt.style.context('seaborn-whitegrid'):    
    fig = plt.figure(figsize=(14, 3.54), dpi=96)
    layout = (1, 1)
    ax = plt.subplot2grid(layout, (0, 0))

    grouped_mean.plot.bar(ax=ax, rot=0)

One approach could be to group `hr` values together according to the different levels of the target:

In [39]:
disc = KBinsDiscretizer(n_bins=4, encode='ordinal')
bins = disc.fit_transform(grouped_mean)
grouped_mean['bins'] = bins

In [ ]:
bin_values = [0, 1, 2, 3]
color_list = ['#74a9cf', '#fc8d59', '#9e9ac8', '#a1d99b']
b2c = dict(zip(bin_values, color_list))

with plt.style.context('seaborn-whitegrid'):    
    fig = plt.figure(figsize=(14, 3.54), dpi=96)
    layout = (1, 1)
    ax = plt.subplot2grid(layout, (0, 0))

    grouped_mean['cnt'].plot.bar(ax=ax, rot=0, color=[b2c[i] for i in grouped_mean['bins']])

Going one step further, we could examine not only the mean of the target per `hr` value but also other characteristics of the distribution. As an example, while `hr=7` and `hr=12` are lumbed into the same category, they have quite different distributions, which may or may not be an issue:

In [ ]:
with plt.style.context('seaborn-whitegrid'):    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 3.54))

    to_group[to_group['hr']==7]['cnt'].plot.hist(ax=ax1, bins=30, alpha=0.5)
    to_group[to_group['hr']==12]['cnt'].plot.hist(ax=ax2, bins=30, alpha=0.5)

To take more aspects of the target's distribution into account, the `eensight.features.encode.TargetClusterEncoder` clusters the different values of a categorical feature according to the mean, standard deviation, skewness and [Wasserstein distance](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.wasserstein_distance.html) of the distribution of the corresponding target's values. 

In [42]:
enc = TargetClusterEncoder(feature='hr', max_n_categories=4)

In [43]:
enc = enc.fit(X, y)

We can update the `bins` column based on the encoder's mapping between values of `hr` and clusters:

In [44]:
grouped_mean['bins'] = grouped_mean.index.map(lambda x: enc.mapping_[x])

... and plot the new features again:

In [ ]:
bin_values = [0, 1, 2, 3]
color_list = ['#74a9cf', '#fc8d59', '#9e9ac8', '#a1d99b']
b2c = dict(zip(bin_values, color_list))

with plt.style.context('seaborn-whitegrid'):    
    fig = plt.figure(figsize=(14, 3.54), dpi=96)
    layout = (1, 1)
    ax = plt.subplot2grid(layout, (0, 0))

    grouped_mean['cnt'].plot.bar(ax=ax, rot=0, color=[b2c[i] for i in grouped_mean['bins']])

In order to judge the value of the encoding scheme, we will compare three cases:
1. A random forest regressor is fitted on data without encoding
2. A random forest regressor is fitted on data with encoding based only on mean target values
3. A random forest regressor is fitted on data with encoding based on target value distribution

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X.drop('dteday', axis=1), y, 
                                                    test_size=0.33, shuffle=True)

In [57]:
scores = []

#### 1. A random forest regressor is fitted on data without encoding

In [58]:
regr = RandomForestRegressor(n_estimators=500, criterion='mse', 
                             min_samples_leaf=1, max_features='auto', 
                             bootstrap=True, max_samples=0.6)

In [59]:
regr = regr.fit(X_train, y_train)

In [ ]:
mse = mean_squared_error(y_test, regr.predict(X_test))
scores.append(mse)
print(f'Mean squared out-of-sample error: {mse}')

#### 2. A random forest regressor is fitted on data with encoding based only on mean target values

In [61]:
to_group = pd.concat((y_train, X_train['hr']), axis=1)
grouped_mean = to_group.groupby('hr').mean()

In [62]:
enc = KBinsDiscretizer(n_bins=4, encode='ordinal')
bins = enc.fit_transform(grouped_mean)
grouped_mean['bins'] = bins.astype(int)
mapping = grouped_mean['bins'].to_dict()

In [63]:
X_train_ = X_train.copy()
X_test_ = X_test.copy()

X_train_['hr'] = X_train_['hr'].map(lambda x: mapping[x])
X_test_['hr'] = X_test_['hr'].map(lambda x: mapping[x])

In [64]:
regr = regr.fit(X_train_, y_train)

In [ ]:
mse = mean_squared_error(y_test, regr.predict(X_test_))
scores.append(mse)
print(f'Mean squared out of sample error: {mse}')

#### 3. A random forest regressor is fitted on data with encoding based on target value distribution

In [66]:
X_train_ = X_train.copy()
X_test_ = X_test.copy()

enc = TargetClusterEncoder(feature='hr', max_n_categories=4)
X_train_['hr'] = enc.fit_transform(X_train_, y_train)
X_test_['hr'] = enc.transform(X_test_)

In [67]:
regr = regr.fit(X_train_, y_train)

In [ ]:
mse = mean_squared_error(y_test, regr.predict(X_test_))
scores.append(mse)
print(f'Mean squared out of sample error: {mse}')

We lose predictive power by reducing the cardinality of the `hr` feature, but we get better results when taking into consideration the overall distribution of the target's data.

#### Conditional effect on target

Furthermore, one may be interested in clustering the `hr` feature taking into account the `weekday` feature: how similar are the target's values in two distinct values of `hr` given similar values for `weekday`?

In this case, the encoder first stratifies the categorical feature `hr` into groups with similar values of `weekday`, and then examines the relationship between the categorical feature's values and the corresponding values of the target.

The stratification is carried out by a `sklearn.tree.DecisionTreeRegressor` model that first fits the `stratify_by` features (here `weekday`) on the target values, and then uses the tree's leaf nodes as groups. 

Only the mean of the target's values per group is taken into account when deriving the clusters. 

The parameter `min_samples_leaf` defines the minimum number of samples required to be at a leaf node of the decision tree model. Note that the actual number that will be passed to the tree model is `min_samples_leaf` multiplied by the number of unique values in the categorical feature to transform.   

In [69]:
enc = TargetClusterEncoder(feature='hr', max_n_categories=4, stratify_by='weekday',
                           min_samples_leaf=15)

In [70]:
X_train_ = X_train.copy()
X_test_ = X_test.copy()

X_train_['hr'] = enc.fit_transform(X_train_, y_train)
X_test_['hr'] = enc.transform(X_test_)

In [71]:
regr = regr.fit(X_train_, y_train)

In [ ]:
mse = mean_squared_error(y_test, regr.predict(X_test_))
scores.append(mse)
print(f'Mean squared out of sample error: {mse}')

We get even better results while still encoding the `hr` feature with only 4 levels.

In [ ]:
np.unique(list(enc.mapping_.values()))

In [ ]:
with plt.style.context('seaborn-whitegrid'):    
    fig = plt.figure(figsize=(14, 3.54), dpi=96)
    layout = (1, 1)
    ax = plt.subplot2grid(layout, (0, 0))

    pd.Series(scores, index=['All 24 values', '4 values - only target mean',  
                             '4 values - target distribution', '4 values - conditional effect']
    ).plot.bar(ax=ax, rot=0)

An alternative way to understand the `stratify_by` effect is to cluster the "day of week" feature into 3 categories by taking into account the impact of the "hour of the day" feature:

In [79]:
enc = TargetClusterEncoder(feature='weekday', max_n_categories=3, stratify_by='hr',
                           min_samples_leaf=15)

In [80]:
enc = enc.fit(X, y)

In [81]:
to_group = pd.concat((y, X['weekday']), axis=1)
grouped_mean = to_group.groupby('weekday').mean()
grouped_mean['bins'] = grouped_mean.index.map(lambda x: enc.mapping_[x])

In [ ]:
bin_values = [0, 1, 2]
color_list = ['#74a9cf', '#fc8d59', '#9e9ac8']
b2c = dict(zip(bin_values, color_list))

with plt.style.context('seaborn-whitegrid'):    
    fig = plt.figure(figsize=(14, 2.54), dpi=96)
    layout = (1, 1)
    ax = plt.subplot2grid(layout, (0, 0))

    grouped_mean['cnt'].plot.bar(ax=ax, rot=0, color=[b2c[i] for i in grouped_mean['bins']])

The mean values provide absolutely no information on why the days were grouped in the way they did. However, it is easy to understand the result if we consider that when the encoder groups days stratified by hours, it actually groups the daily profiles of the target's value per hour: 

In [83]:
profiles = pd.concat((y, X[['weekday', 'hr']]), axis=1).groupby(['weekday', 'hr'])  \
             .mean().unstack()

In [ ]:
with plt.style.context('seaborn-whitegrid'):    
    fig = plt.figure(figsize=(14, 3.5), dpi=96)
    layout = (1, 1)
    ax = plt.subplot2grid(layout, (0, 0))
    
    for day, values in profiles.iterrows():
        values.plot(ax=ax, color=b2c[enc.mapping_[day]])

### `eensight.features.encode.CategoricalEncoder` 

Encodes a categorical feature by encaptulating all the aforementioned categorical encoders so far.

    Parameters
    ----------
    feature : str
        The name of the categorical feature to transform. This encoder operates on 
        a single feature.
    max_n_categories : int (default=None)
        The maximum number of categories to produce.
    stratify_by : str or list of str (default=None)
        If not None, the encoder will first stratify the categorical feature into
        groups that have similar values of the features in `stratify_by`, and then
        cluster based on the relationship between the categorical feature and the
        target.
    excluded_categories : str or list of str (default=None)
        The names of the categories to be excluded from the clustering process. These
        categories will stay intact by the encoding process, so they cannot have the
        same values as the encoder's results (the encoder acts as an ``OrdinalEncoder``
        in the sense that the feature is converted into a column of integers 0 to
        n_categories - 1).
    unknown_value : int, default=None
        This parameter will set the encoded value of unknown categories. It has to
        be distinct from the values used to encode any of the categories in `fit`.
        If None, the value `-1` is used.
    min_samples_leaf : int, default=1
        The minimum number of samples required to be at a leaf node of the decision
        tree model that is used for stratifying the categorical feature if `stratify_by`
        is not None. The actual number that will be passed to the tree model is
        `min_samples_leaf` multiplied by the number of unique values in the categorical
        feature to transform.
    max_features : int, float or {"auto", "sqrt", "log2"}, default=None
        The number of features to consider when looking for the best split:
        - If int, then consider `max_features` features at each split of the decision tree.
        - If float, then `max_features` is a fraction and
        `int(max_features * n_features)` features are considered at each
        split.
        - If "auto", then `max_features=n_features`.
        - If "sqrt", then `max_features=sqrt(n_features)`.
        - If "log2", then `max_features=log2(n_features)`.
        - If None, then `max_features=n_features`.
    random_state : int, RandomState instance or None, default=None
        Controls the randomness of the estimator. To obtain a deterministic behaviour
        during fitting, ``random_state`` has to be fixed to an integer.
    encode_as : str {'onehot', 'ordinal'}, default='onehot'
        Method used to encode the transformed result.
        onehot
            Encode the transformed result with one-hot encoding
            and return a dense array.
        ordinal
            Encode the transformed result as integer values.

If `max_n_categories` is not None and the number of distinct values of the categorical feature is larger than the `max_n_categories`, the `eensight.features.encode.TargetClusterEncoder` will be called. 

If `encode_as = 'onehot'`, the result comes from a `eensight.features.encode.TargetClusterEncoder` + `eensight.features.encode.SafeOneHotEncoder` pipeline, otherwise from `eensight.features.encode.TargetClusterEncoder` + `eensight.features.encode.SafeOrdinalEncoder`.



In [102]:
max_n_categories = 10

In [ ]:
enc = CategoricalEncoder(feature='weekday', max_n_categories=max_n_categories, 
                         encode_as='onehot')
feature_trf = enc.fit_transform(X, y)
feature_trf[:5]

In [104]:
assert min(X['weekday'].nunique(), max_n_categories) == enc.n_features_out_

In [ ]:
enc = CategoricalEncoder(feature='weekday', max_n_categories=10, encode_as='ordinal')
feature_trf = enc.fit_transform(X, y)
feature_trf[:5]

In [106]:
assert min(X['weekday'].nunique(), max_n_categories) == np.unique(feature_trf).size

In [108]:
max_n_categories = 3

In [ ]:
enc = CategoricalEncoder(feature='weekday', max_n_categories=max_n_categories, 
                         encode_as='onehot')
feature_trf = enc.fit_transform(X, y)
feature_trf[:5]

In [110]:
assert min(X['weekday'].nunique(), max_n_categories) == enc.n_features_out_

In [ ]:
enc = CategoricalEncoder(feature='weekday', max_n_categories=max_n_categories, 
                         encode_as='ordinal')

feature_trf = enc.fit_transform(X, y)
feature_trf[:5]

In [112]:
assert min(X['weekday'].nunique(), max_n_categories) == np.unique(feature_trf).size

## Encoding pairwise interactions between categorical features

### `eensight.features.encode.ICatEncoder` 

Encodes the interaction between two categorical features

    Parameters
    ----------
    encoder_left : eensight.features.encode.CategoricalEncoder
        The encoder for the first of the two features
    encoder_right : eensight.features.encode.CategoricalEncoder
        The encoder for the second of the two features

**Note**: Both encoders should have the same `encode_as` parameter.
If one or both of the encoders is already fitted, it will not be re-fitted during `fit` or `fit_transform`.


If `encode_as = 'onehot'`, it returns the tensor product of the results of the two encoders. The tensor product combines row-per-row the results from the first and the second encoder as follows:

![tensor product](images/tensor.png)

A small example of the tensor product function:

In [ ]:
a = np.array([1, 10]).reshape(1, -1)
b = np.array([10, 20, 30]).reshape(1, -1)

tensor_product(a, b)

The easiest way to demonstate it is by combining hours of day and days of week into hours of week:

In [ ]:
enc_day = CategoricalEncoder(feature='weekday', encode_as='onehot')
feature_trf = enc_day.fit_transform(X)
feature_trf.shape

In [ ]:
enc_hour = CategoricalEncoder(feature='hr', encode_as='onehot')
feature_trf = enc_hour.fit_transform(X)
feature_trf.shape

In [116]:
enc = ICatEncoder(enc_day, enc_hour)

In [ ]:
print(f'We expect {7*24} columns in the interactions')

In [ ]:
feature_trf = enc.fit_transform(X)
feature_trf.shape

Make sure it is a proper one-hot array:

In [121]:
assert np.array_equal(feature_trf.sum(axis=1), np.ones(len(X)))

If `encode_as = 'ordinal'`, it returns the combinations of the encoders' results, where each combination is a string with `:` between the two values:

In [ ]:
enc_day = CategoricalEncoder(feature='weekday', encode_as='ordinal')
enc_hour = CategoricalEncoder(feature='hr', encode_as='ordinal')
enc = ICatEncoder(enc_day, enc_hour)

feature_trf = enc.fit_transform(X)
feature_trf

In [123]:
assert np.unique(feature_trf).size == 168

## Encoding numerical features

### `eensight.features.encode.SplineEncoder` 

Encodes a numerical feature by generating univariate B-spline bases.
It generates a new feature matrix consisting of `n_splines=n_knots + degree - 1` 
spline basis functions (B-splines) of polynomial order=`degree` for each feature.

    Parameters
    ----------
    feature : str
        The name of the column to encode.
    n_knots : int, default=5
        Number of knots of the splines if `knots` equals one of {'uniform', 'quantile'}.
        Must be larger or equal 2. Ignored if `knots` is array-like.
    degree : int, default=3
        The polynomial degree of the spline basis. Must be a non-negative integer.
    strategy : {'uniform', 'quantile'} or array-like of shape (n_knots, n_features),
        default='quantile'
        Set knot positions such that first knot <= features <= last knot.
        - If 'uniform', `n_knots` number of knots are distributed uniformly
        from min to max values of the features (each bin has the same width).
        - If 'quantile', they are distributed uniformly along the quantiles of
        the features (each bin has the same number of observations).
        - If an array-like is given, it directly specifies the sorted knot
        positions including the boundary knots. Note that, internally,
        `degree` number of knots are added before the first knot, the same
        after the last knot.
    extrapolation : {'error', 'constant', 'linear', 'continue'}, default='constant'
        If 'error', values outside the min and max values of the training features
        raises a `ValueError`. If 'constant', the value of the splines at minimum
        and maximum value of the features is used as constant extrapolation. If
        'linear', a linear extrapolation is used. If 'continue', the splines are
        extrapolated as is, i.e. option `extrapolate=True` in `scipy.interpolate.BSpline`.
    include_bias : bool, default=False
        If False, then the last spline element inside the data range of a feature
        is dropped. As B-splines sum to one over the spline basis functions for each
        data point, they implicitly include a bias term.
    order : {'C', 'F'}, default='C'
        Order of output array. 'F' order is faster to compute, but may slow
        down subsequent estimators.

For this, we will create some synthetic data:

In [138]:
def f(x):
    return 10 + (x * np.sin(x))

In [139]:
x_support = np.linspace(0, 15, 100)
y_support = f(x_support)

x_train = np.sort(np.random.choice(x_support[15:-15], size=25, replace=False))
y_train = f(x_train)

In [140]:
X_train = pd.DataFrame(data=x_train, columns=['input'])
X_support = pd.DataFrame(data=x_support, columns=['input'])

In [ ]:
with plt.style.context('seaborn-whitegrid'):    
    fig = plt.figure(figsize=(14, 3.5), dpi=96)
    layout = (1, 1)
    ax = plt.subplot2grid(layout, (0, 0))
    
    ax.plot(X_support, y_support, label='ground truth', c='#fc8d59')
    ax.plot(X_train, y_train, 'o', label='training points', c='#fc8d59')
    ax.legend(loc='upper left')

**Cubic spline without extrapolation**:

In [ ]:
enc = SplineEncoder(feature='input', n_knots=5, degree=3, strategy="quantile", 
                        extrapolation="constant", include_bias=True,) 

model = make_pipeline(enc, LinearRegression(fit_intercept=False))
model.fit(X_train, y_train)
pred = model.predict(X_support)

with plt.style.context('seaborn-whitegrid'):    
    fig = plt.figure(figsize=(14, 3.5), dpi=96)
    layout = (1, 1)
    ax = plt.subplot2grid(layout, (0, 0))
    
    ax.plot(X_support, y_support, label='ground truth', c='#fc8d59')
    ax.plot(X_train, y_train, 'o', label='training points', c='#fc8d59')
    ax.plot(X_support, pred, label='spline approximation')
    ax.legend(loc='upper left')

**With linear extrapolation**:

In [ ]:
enc = SplineEncoder(feature='input', n_knots=5, degree=3, strategy="quantile", 
                        extrapolation="linear", include_bias=True,) 

model = make_pipeline(enc, LinearRegression(fit_intercept=False))
model.fit(X_train, y_train)
pred = model.predict(X_support)

with plt.style.context('seaborn-whitegrid'):    
    fig = plt.figure(figsize=(14, 3.5), dpi=96)
    layout = (1, 1)
    ax = plt.subplot2grid(layout, (0, 0))
    
    ax.plot(X_support, y_support, label='ground truth', c='#fc8d59')
    ax.plot(X_train, y_train, 'o', label='training points', c='#fc8d59')
    ax.plot(X_support, pred, label='spline approximation')
    ax.legend(loc='upper left')

## Encoding pairwise interactions between numerical features

### `eensight.features.encode.ISplineEncoder` 

Encode the interaction between two spline-encoded numerical features

    Parameters
    ----------
    encoder_left : eensight.features.encode.SplineEncoder
        The encoder for the first of the two features
    encoder_right : eensight.features.encode.SplineEncoder
        The encoder for the second of the two features

**Note**: If one or both of the encoders is already fitted, it will not be 
          re-fitted during `fit` or `fit_transform`.

Generate the [“Friedman #1”](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_friedman1.html) regression problem:

In [144]:
X, y = make_friedman1(n_samples=5000, n_features=5, noise=0.2)

In [ ]:
X = pd.DataFrame(data=X, columns=[f'x_{i}' for i in range(5)])
y = pd.Series(data=y, index=X.index)
X.head()

In [146]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, shuffle=True)

In [147]:
enc_1 = SplineEncoder(feature='x_0', n_knots=5, degree=3, strategy="quantile", 
                        extrapolation="constant", include_bias=False,)
enc_2 = SplineEncoder(feature='x_1', n_knots=5, degree=3, strategy="quantile", 
                        extrapolation="constant", include_bias=False,)
enc_3 = SplineEncoder(feature='x_2', n_knots=5, degree=3, strategy="quantile", 
                        extrapolation="constant", include_bias=False,)
enc_4 = SplineEncoder(feature='x_3', n_knots=5, degree=3, strategy="quantile", 
                        extrapolation="constant", include_bias=False,)
enc_5 = SplineEncoder(feature='x_4', n_knots=5, degree=3, strategy="quantile", 
                        extrapolation="constant", include_bias=False,)

interact = ISplineEncoder(enc_1, enc_2)

In [148]:
pipeline = Pipeline([
    ('features', FeatureUnion([
                    ('inter', interact),
                    ('enc_3', enc_3),
                    ('enc_4', enc_4),
                    ('enc_5', enc_5)
    
                ])
    ),
    ('regression', LinearRegression(fit_intercept=True))
])


In [149]:
pipeline = pipeline.fit(X_train, y_train)

The root mean squared error is very close to the noise that was injected in the data (0.2):

In [ ]:
print('Root mean squared out-of-sample error: ' 
      f'{mean_squared_error(np.array(y_test), pipeline.predict(X_test), squared=False)}'
)

Linear interations are also supported through `ProductEncoder`:

### `eensight.features.encode.ProductEncoder` 

Encode the interaction between two linear numerical features

    Parameters
    ----------
    encoder_left : eensight.features.encode.IdentityEncoder
        The encoder for the first of the two features
    encoder_right : eensight.features.encode.IdentityEncoder
        The encoder for the second of the two features

**Note**: If one or both of the encoders is already fitted, it will not be
        re-fitted during `fit` or `fit_transform`.

In [151]:
enc_1 = IdentityEncoder(feature='x_0', include_bias=False,)
enc_2 = IdentityEncoder(feature='x_1', include_bias=False,)

interact = ProductEncoder(enc_1, enc_2)

This interaction is practically an element-wise multiplication of the two features: 

In [152]:
assert np.allclose(interact.fit_transform(X).squeeze(), X[['x_0', 'x_1']].prod(axis=1))

## Encoding pairwise interactions between categorical and numerical features

We will use a different dataset this time: a hypothetical study of weight loss for 900 participants in a year-long study of 3 different exercise programs, a jogging program, a swimming program, and a reading program which serves as a control activity. Variables include:

- **loss**: weight loss (continuous), positive = weight loss, negative scores = weight gain
- **hours**: hours spent exercising (continuous)
- **effort**: effort during exercise (continuous), 0 = minimal physical effort and 50 = maximum effort
- **prog**: exercise program (categorical)
    - jogging=1
    - swimming=2
    - reading=3
- **gender**: participant gender (binary)
    - male=1
    - female=2

In [ ]:
path = 'play_data/weight_loss/data.csv'
data = pd.read_csv(path)
data = data.drop('id', axis=1)
data.head()

In [5]:
data_train, data_test = train_test_split(data, train_size=0.5, shuffle=True)

We want to identify the way `prog` (categorical feature) and `hours` (numerical feature) interact: `loss ~ prog:hours`. 

First, we can explore the case where we split all the data by `prog` and fit a `loss ~ hours` model to each group:

In [6]:
models = {}

for group, grouped_data in data_train.groupby('prog'):
    model = LinearRegression(fit_intercept=True).fit(grouped_data[['hours']], 
                                                     grouped_data['loss'])
    models[group] = model

In [ ]:
color_list = ['#74a9cf', '#fc8d59', '#9e9ac8']

with plt.style.context('seaborn-whitegrid'):    
    fig = plt.figure(figsize=(14, 4.5), dpi=96)
    layout = (1, 1)
    ax = plt.subplot2grid(layout, (0, 0))
    
    resid = []
    for i, (group, grouped_data) in enumerate(data_test.groupby('prog')):
        pred = models[group].predict(grouped_data[['hours']])
        resid.append(grouped_data['loss'].values - pred)
        
        ax.plot(grouped_data['hours'], pred, '.', label=f'prog: {group}', c=color_list[i])
        ax.plot(grouped_data['hours'], grouped_data['loss'], 'o', c=color_list[i], alpha=0.2)
        
    ax.legend(loc='upper left')
    

In [ ]:
print(f'Mean squared error: {np.mean(np.concatenate(resid)**2)}')

The same result can be achieved by first encoding the `prog` feature in one-hot form and then taking the tensor product between its encoding and the `hours` feature. In this case, an intercept must be added directly to the `hours` feature, so that it is possible to model a different intercept for each categorical feature's level:   

In [ ]:
enc_cat = CategoricalEncoder(feature='prog', encode_as='onehot')
feature_cat = enc_cat.fit_transform(data_train)
feature_cat.shape

In [ ]:
features = tensor_product(feature_cat, add_constant(data_train['hours']))
features.shape

In [11]:
model = LinearRegression(fit_intercept=True).fit(features, data_train['loss'])

In [12]:
feature_cat = enc_cat.transform(data_test)
features = tensor_product(feature_cat, add_constant(data_test['hours']))
pred = model.predict(features)

In [ ]:
resid = data_test['loss'].values - pred
print(f'Mean squared error: {np.mean(resid**2)}')

In [ ]:
color_list = ['#74a9cf', '#fc8d59', '#9e9ac8']

with plt.style.context('seaborn-whitegrid'):    
    fig = plt.figure(figsize=(14, 4.5), dpi=96)
    layout = (1, 1)
    ax = plt.subplot2grid(layout, (0, 0))
    
    for i in range(enc_cat.n_features_out_):
        mask = feature_cat[:, i]==1
        ax.plot(data_test['hours'][mask], pred[mask], '.', label=f'prog: {i}', 
                c=color_list[i])
        ax.plot(data_test['hours'][mask], data_test['loss'][mask], 'o', 
                c=color_list[i], alpha=0.3)
        
    ax.legend(loc='upper left')

The conclusion here is that for the case of one categorical and one linear numerical feature, we can model the interaction by first encoding the categorical feature in one-hot form and then taking the tensor product between this encoding and the numerical feature. 

This is supported by `eensight.features.encode.ICatLinearEncoder`.

### `eensight.features.encode.ICatLinearEncoder` 

Encodes the interaction between one categorical and one linear numerical feature

    Parameters
    ----------
    encoder_cat : eensight.features.encode.CategoricalEncoder
        The encoder for the categorical feature. It must encode features in an one-hot
        form.
    encoder_num : eensight.features.encode.IdentityEncoder
        The encoder for the numerical feature. The  encoder should have 
        `include_bias=True`. This is necessary so that so that it is possible 
         to model a different intercept for each categorical feature's level.
        

In [15]:
enc_cat = CategoricalEncoder(feature='prog', encode_as='onehot')
enc_num = IdentityEncoder(feature='hours', include_bias=True)

enc = ICatLinearEncoder(encoder_cat=enc_cat, encoder_num=enc_num)

In [16]:
features = enc.fit_transform(data_train)
model = LinearRegression(fit_intercept=False).fit(features, data_train['loss'])

In [17]:
features = enc.transform(data_test)
pred = model.predict(features)

In [ ]:
resid = data_test['loss'].values - pred
print(f'Mean squared error: {np.mean(resid**2)}')

Next, we want to also encode the `hours` feature with splines so that to capture potential non-linearities (although in this specific dataset there are no non-linearities). 

A ***first split, then encode*** strategy looks like this:

In [19]:
models = {}
encoders = {}

for group, grouped_data in data_train.groupby('prog'):
    enc = SplineEncoder(feature='hours', n_knots=3, degree=2, 
                                    strategy="quantile", 
                                    extrapolation="constant", 
                                    include_bias=True,) 
    features = enc.fit_transform(grouped_data)
    model = LinearRegression(fit_intercept=False).fit(features, grouped_data['loss'])
    models[group] = model
    encoders[group] = enc

In [ ]:
color_list = ['#74a9cf', '#fc8d59', '#9e9ac8']

with plt.style.context('seaborn-whitegrid'):    
    fig = plt.figure(figsize=(14, 4.5), dpi=96)
    layout = (1, 1)
    ax = plt.subplot2grid(layout, (0, 0))
    
    resid = []
    for i, (group, grouped_data) in enumerate(data_test.groupby('prog')):
        features = encoders[group].transform(grouped_data)
        pred = models[group].predict(features)
        resid.append(grouped_data['loss'].values - pred)
        
        ax.plot(grouped_data['hours'], pred, '.', label=f'prog: {group}', c=color_list[i])
        ax.plot(grouped_data['hours'], grouped_data['loss'], 'o', c=color_list[i], alpha=0.2)
        
    ax.legend(loc='upper left')

In [ ]:
print(f'Mean squared error: {np.mean(np.concatenate(resid)**2)}')

The `eensight` implements the **first split, then encode** strategy in `ICatSplineEncoder`:

### `eensight.features.encode.ICatSplineEncoder` 

Encodes the interaction between one categorical and one spline-encoded numerical
feature. This encoder can also work with a cyclical numerical feature.

    Parameters
    ----------
    encoder_cat : eensight.features.encode.CategoricalEncoder
        The encoder for the categorical feature. It must encode features in
        an one-hot form.
    encoder_num : eensight.features.encode.SplineEncoder or
        eensight.features.encode.CyclicalEncoder
        The encoder for the numerical feature.

**Notes**: 

- If the categorical encoder is already fitted, it will not be re-fitted during `fit` or `fit_transform`. 

- The numerical encoder will always be fitted (one encoder per level of categorical feature).

- If the numerical encoder is a `SplineEncoder`, it should have `include_bias=True`. This is necessary so that so that it is possible to model a different intercept for each categorical feature's level.

In [24]:
enc_cat = CategoricalEncoder(feature='prog', encode_as='onehot')

enc_num = SplineEncoder(feature='hours', n_knots=3, degree=2, 
                                    strategy="quantile", 
                                    extrapolation="constant", 
                                    include_bias=True,) 

In [25]:
enc = ICatSplineEncoder(encoder_cat=enc_cat, encoder_num=enc_num)

In [26]:
features = enc.fit_transform(data_train)
model = LinearRegression(fit_intercept=False).fit(features, data_train['loss'])

In [27]:
features = enc.transform(data_test)
pred = model.predict(features)

In [ ]:
resid = data_test['loss'].values - pred
print(f'Mean squared error: {np.mean(resid**2)}')

We can further explore the **first split, then encode** strategy by predicting: `loss ~ gender:prog:hours`. 


The ***first split, then encode*** strategy is equivalent to:

In [29]:
models = {}
encoders = {}

for gender, level_1_data in data_train.groupby('gender'):
    for prog, grouped_data in level_1_data.groupby('prog'):
        enc = SplineEncoder(feature='hours', n_knots=3, degree=2, 
                                    strategy="quantile", 
                                    extrapolation="constant", 
                                    include_bias=True,)
        features = enc.fit_transform(grouped_data)
        encoders[(gender, prog)] = enc
        
        model = LinearRegression(fit_intercept=False).fit(features, grouped_data['loss'])
        models[(gender, prog)] = model
        

In [32]:
pred = None
for gender, level_1_data in data_test.groupby('gender'):
    for prog, grouped_data in level_1_data.groupby('prog'):
        features = encoders[(gender, prog)].transform(grouped_data)
        pred = pd.concat((pred, pd.Series(models[(gender, prog)].predict(features), 
                                          index=grouped_data.index)))
        
pred = pred.reindex(data_test.index)

In [ ]:
resid = data_test['loss'].values - pred
print(f'Mean squared error: {np.mean(resid**2)}')

This is equivalent to:

In [35]:
enc_cat = CategoricalEncoder(feature='prog', encode_as='onehot')
enc_cat = enc_cat.fit(data_train)

enc_num = SplineEncoder(feature='hours', n_knots=3, degree=2, strategy="quantile", 
                        extrapolation="constant", include_bias=True,) 

In [36]:
models = {}
encoders = {}

for gender, grouped_data in data_train.groupby('gender'):
    enc = ICatSplineEncoder(encoder_cat=enc_cat, encoder_num=enc_num)
    features = enc.fit_transform(grouped_data)
    
    model = LinearRegression(fit_intercept=False).fit(features, grouped_data['loss'])
    models[gender] = model
    encoders[gender] = enc

In [37]:
pred = None
for gender, grouped_data in data_test.groupby('gender'):
    features = encoders[gender].transform(grouped_data)
    pred = pd.concat((pred, pd.Series(models[gender].predict(features), 
                                      index=grouped_data.index)))
        
pred = pred.reindex(data_test.index)

In [ ]:
resid = data_test['loss'].values - pred
print(f'Mean squared error: {np.mean(resid**2)}')

Despite its name, the `ICatSplineEncoder` endoder can work also when the numerical feature is encoded with `CyclicalFeatures`:

In [ ]:
data = pd.DataFrame(index=pd.date_range(start='1/1/2018', end='31/12/2019', freq='D'))
data['weekday'] = data.index.dayofweek < 5

data.head()

In [54]:
enc_cat = CategoricalEncoder(feature='weekday', encode_as='onehot')
enc_cat = enc_cat.fit(data)

In [55]:
enc_num = CyclicalFeatures(period=365, fourier_order=3)

In [56]:
enc = ICatSplineEncoder(encoder_cat=enc_cat, encoder_num=enc_num)

In [ ]:
features = enc.fit_transform(data)
features.shape

For the case of cyclical data, the **first split then encode** and the **first encode then split** strategies are equivalent, because the encoding uses only the info of each row and not any other value from the same column:

In [58]:
features_num = CyclicalFeatures(period=365, fourier_order=3).fit_transform(data)
features_cat = CategoricalEncoder(feature='weekday', encode_as='onehot').fit_transform(data)

In [ ]:
np.array_equal(features, tensor_product(features_cat, features_num))

Combining a `CategoricalEncoder` with a `CyclicalFeatures` is one way to create features of custom seasonalities very similarly to how the [Prophet](https://github.com/facebook/prophet) python library does it: https://facebook.github.io/prophet/docs/seasonality,_holiday_effects,_and_regressors.html#specifying-custom-seasonalities 

------------------